In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout


2024-01-14 02:37:40.083434: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Tạo fake data
np.random.seed(0)
num_samples = 1000

seconds = np.arange(1, num_samples + 1)
acceleration = np.random.uniform(0, 1, num_samples)
angle_hand = np.random.uniform(0, 1, num_samples)
angle_neck = np.random.uniform(0, 1, num_samples)
angle_right_leg = np.random.uniform(0, 1, num_samples)
angle_left_leg = np.random.uniform(0, 1, num_samples)
score = np.random.uniform(0, 100, num_samples)

data = pd.DataFrame({
    'Seconds': seconds,
    'Acceleration': acceleration,
    'AngleHand': angle_hand,
    'AngleNeck': angle_neck,
    'AngleRightLeg': angle_right_leg,
    'AngleLeftLeg': angle_left_leg,
    'Score': score
})

# Lưu vào file CSV
data.to_csv('fake_data.csv', index=False)

# Đọc dữ liệu từ file CSV
read_data = pd.read_csv('fake_data.csv')

# Chuẩn hóa dữ liệu
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(read_data.drop('Score', axis=1))

# Tạo dữ liệu đầu vào và đầu ra cho mô hình
X = data_scaled
y = read_data['Score'].values

# Chia thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape dữ liệu để phù hợp với input của mô hình LSTM
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Xây dựng mô hình BI LSTM
# Xây dựng mô hình BI LSTM với nhiều layer
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu', return_sequences=True), input_shape=(1, X_train.shape[2])))
model.add(Dropout(0.2))  # Dropout layer để tránh overfitting

model.add(Bidirectional(LSTM(50, activation='relu', return_sequences=True)))
model.add(Dropout(0.2))

model.add(Bidirectional(LSTM(50, activation='relu')))
model.add(Dropout(0.2))

model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')


# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=2)
